# User Authentication using OAuth2 and APIM fronting Azure OpenAI

In [5]:
tenant_id = "f69a7636-4db8-498c-8ff6-bc7f1aafcec0" # my MCAPS tenant
client_id = "c2933b1e-ba50-4177-ac78-11a564edf49d" # web-lliybm23riihe-docker client id
app_endpoint = "https://web-lliybm23riihe-docker.azurewebsites.net"


In [ ]:
%pip install msal
%pip install openai
%pip install requests
%pip install logging
%pip install json

##  Create a device flow to get the access token

In [11]:
# Step 1: Install MSAL
# pip install msal

import msal

# Step 2: Configure MSAL
authority = 'https://login.microsoftonline.com/f69a7636-4db8-498c-8ff6-bc7f1aafcec0'
# below is the managed identity for APIM
scopes = ["c2933b1e-ba50-4177-ac78-11a564edf49d"] # , "User.Read"]

# Create a public client application
app = msal.PublicClientApplication(client_id, authority=authority)

# Step 3: Initiate Device Flow
flow = app.initiate_device_flow(scopes=scopes)
if 'user_code' not in flow:
    raise ValueError("Failed to create device flow. Check your client ID and authority.")

print(flow['message'])

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AFEHWS4UV to authenticate.


In [8]:
# Step 4: Acquire Token
result = app.acquire_token_by_device_flow(flow)
if 'access_token' in result:
    print("Access token acquired.")
else:
    raise ValueError("Failed to acquire token. Error: {}".format(result.get('error_description')))

access_token = result['access_token']

# Step 5: Validate Token (Optional)
# You can decode the token and check the audience if needed
import jwt

decoded_token = jwt.decode(access_token, options={"verify_signature": False})
audience = decoded_token.get('aud')
# if audience != 'api://your-apim-instance':
#     raise ValueError("Token audience does not match the expected audience.")

print(decoded_token)
print(audience )

Access token acquired.
{'aud': '00000003-0000-0000-c000-000000000000', 'iss': 'https://sts.windows.net/f69a7636-4db8-498c-8ff6-bc7f1aafcec0/', 'iat': 1730743151, 'nbf': 1730743151, 'exp': 1730747433, 'acct': 1, 'acr': '1', 'aio': 'AXQAi/8YAAAAMTyWm8ka3it8xaOGmxe/Qe1ayMRbRzGZUDGPXPuZCiWoM0o8xnbojAUDf80v131ydXIPdPk8tllBYe8adUaX5QaAGc9FqiMUndpBguTYpTQxGhwUxvFLLzF2ZXQSvj3nRa4wxh8A+3atZ0K4vVJ2oA==', 'altsecid': '5::10033FFFA87156C9', 'amr': ['pwd', 'mfa'], 'app_displayname': 'web-lliybm23riihe-docker', 'appid': 'c2933b1e-ba50-4177-ac78-11a564edf49d', 'appidacr': '0', 'email': 'JOMETZG@microsoft.com', 'idp': 'https://sts.windows.net/72f988bf-86f1-41af-91ab-2d7cd011db47/', 'idtyp': 'user', 'ipaddr': '212.159.20.110', 'name': 'John Metzger', 'oid': 'e3f444c1-3a1c-4b5b-8d49-de0c8df5a5eb', 'platf': '14', 'puid': '10032003C2DF54E7', 'rh': '1.AWMBNnaa9rhNjEmP9rx_Gq_OwAMAAAAAAAAAwAAAAAAAAABjAe5jAQ.', 'scp': 'email Mail.Read openid profile User.Read', 'signin_state': ['kmsi'], 'sub': 'ugDMqgxe-8oVLV

In [23]:
print(audience)
print(access_token)

api://bee929cd-e8ac-4385-b299-e544fd2c476d/newopenai
eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiJhcGk6Ly9iZWU5MjljZC1lOGFjLTQzODUtYjI5OS1lNTQ0ZmQyYzQ3NmQvbmV3b3BlbmFpIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvZjY5YTc2MzYtNGRiOC00OThjLThmZjYtYmM3ZjFhYWZjZWMwLyIsImlhdCI6MTczMDI4NzYwMiwibmJmIjoxNzMwMjg3NjAyLCJleHAiOjE3MzAyOTI0NTIsImFjciI6IjEiLCJhaW8iOiJBWFFBaS84WUFBQUFFcjhyTEJwTTN3bEk1T3J4NmVDN3prOXFvWC9CUEdUSHhoOERWVFp4SDZkQ3BBQVRTK0tzd2NZN1hsR3N0L3lseUoyYTdkVG5ycW43WUpETzRxTHJzY2Z5V1orRHdaODc0eXJwUTRBY2VicFVXdDZUQVM2a3NvNkVWcWwzc2F2UStxZm5Rd28rU1lTcHZaNUUyRG40WUE9PSIsImFtciI6WyJwd2QiLCJtZmEiXSwiYXBwaWQiOiJiZWU5MjljZC1lOGFjLTQzODUtYjI5OS1lNTQ0ZmQyYzQ3NmQiLCJhcHBpZGFjciI6IjAiLCJlbWFpbCI6IkpPTUVUWkdAbWljcm9zb2Z0LmNvbSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzcyZjk4OGJmLTg2ZjEtNDFhZi05MWFiLTJkN2NkMDExZGI0Ny8iLCJpcGFkZHIiOiIyMTIuMTU5LjIwLjExMCIsIm5hbWUiOiJKb2huIE1ldHpnZXIiLCJvaWQiOiJlM2Y0ND

## Test the API using a direct HTTP call

In [ ]:
import requests
import json

url = app_endpoint + "/api/conversation"

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
response = requests.post(url, headers = { 'Authorization': 'Bearer ' + access_token}, json = messages)
print("status code: ", response.status_code)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)

## Test the API using the Azure OpenAI Python SDK

In [ ]:
from openai import AzureOpenAI

messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apim_endpoint,
    api_key=apim_key,
    api_version=openai_api_version        
)
response = client.chat.completions.create(model=openai_deployment_name, messages=messages, extra_headers={"Authorization": "Bearer " + access_token})
print(response.choices[0].message.content)